<a href="https://colab.research.google.com/github/tejaswishetty17/GenAI_Work/blob/main/Adding_Meta_Data_Along_with_the_response.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.5 MB/s eta 0:00:00


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter

In [3]:
import os
os.environ['OPENAI_API_KEY'] = "<YOUR_API_KEY>"


In [7]:
#Step 1: Load and split document
loader = TextLoader("/content/sample_data/langchain_doc.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
chunks = splitter.split_documents(docs)

In [22]:
#Step 2: Embedding and vector store
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")
vectorstore = Chroma.from_documents(chunks, embedding=embedding, persist_directory="./chroma_rag_simple1")

retriver = vectorstore.as_retriever(serach_kwargs={"k":3})

In [23]:
retriver.invoke("what is langchain")

[Document(metadata={'source': '/content/sample_data/langchain_doc.txt'}, page_content='LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around langchain for data extraction and transformation efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides end-to-end support for langchain for data extraction and transformation.'),
 Document(metadata={'source': '/content/sample_data/langchain_doc.txt'}, page_content='LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around langchain for data extraction and transformation efficiently. Whether integrating third-party APIs, wor

In [24]:
#Step 3: Format documents with source metadata

def format_docs(docs):
  context = "\n\n".join(doc.page_content for doc in docs)
  sources = list(set(doc.metadata.get("source", "Unknown") for doc in docs))
  return context, sources

In [25]:
#Step 4: Define the prompt and LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a helful assistant. Use the below context below to answer the question:

Context:
{context}

Question:{question}
""")

chain = prompt | llm | StrOutputParser()

In [26]:
#Step 5: RAG Flow

query = "What is Langchain used for?"

#Get relavant documents
retrieved_docs = retriver.invoke(query)

#Format them
context, sources = format_docs(retrieved_docs)

print("Context: ",context)
print("Sources: ",sources, "\n\n")

#Run through the LLM
answer = chain.invoke({"context":context, "question":query})

#Step 6: Print Results
print("Question", query)
print("Answer", answer)
print("Sources:", sources)

Context:  LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around advanced use cases: coding assistants and research bots efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides end-to-end support for advanced use cases: coding assistants and research bots.

LangChain offers features and tools that make it easier to work with this concept. In particular, the use of modular abstractions, prebuilt utilities, and community-backed best practices ensures that developers can implement solutions around advanced use cases: coding assistants and research bots efficiently. Whether integrating third-party APIs, working with documents, or building agents, LangChain provides end-to-end support for advanced use cases: coding assistants and research bots.

Lang

#Lets do handle data from multiple files

In [31]:
#Step 1: Install required packages (if not already done)
!pip install -q langchain langchain-openai langchain-community chromadb


from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter


file_path = [
    "/content/sample_data/langchain_doc.txt",
    "/content/sample_data/spark_langchain.txt"
]


docs = []

for path in file_path:
  loader = TextLoader(path)
  docs.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 50)
chunks = splitter.split_documents(docs)

#step 2: Embedding and Vectore Store
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")
vectorestore = Chroma.from_documents(chunks, embedding, persist_directory="./chroma_rag_simple3")
retriever = vectorestore.as_retriever(serach_kwargs = {"k":3})

#Step 3: Format documents with source metadata
def format_docs(docs):
  context = "\n\n".join(doc.page_content for doc in docs)
  sources = list(set(doc.metadata.get("source", "Unknown") for doc in docs))
  return context, sources

#Step 4: Define Prompt and LLM
llm = ChatOpenAI(model = "gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Use the context below to answer the question:

Context:
{context}

Question: {question}
""")

chain = prompt | llm | StrOutputParser()

In [34]:
# Step 5: RAG Flow
query = "What is Langchain"

#Get relavent documents
retrieved_docs = retriever.invoke(query)

#Format them
context, sources = format_docs(retrieved_docs)

print("Retrieved Context: ", context)
print("Retrived sources: ", sources, "\n\n\n")

#Run through the LLM
answer = chain.invoke({"context": context, "question": query})

#Step 6:
print("Question:", query)
print("Answer:", answer)
print("Sources:", sources)

Retrieved Context:  What is LangChain?

LangChain is a framework for developing applications powered by large language models (LLMs) like GPT.

What is LangChain?

LangChain is a framework for developing applications powered by large language models (LLMs) like GPT.

What is LangChain?

LangChain is a framework for developing applications powered by large language models (LLMs) like GPT.

What is LangChain?

LangChain is a framework for developing applications powered by large language models (LLMs) like GPT.
Retrived sources:  ['/content/sample_data/spark_langchain.txt'] 



Question: What is Langchain
Answer: LangChain is a framework for developing applications powered by large language models (LLMs) like GPT.
Sources: ['/content/sample_data/spark_langchain.txt']
